In [1]:
import boto3
import datetime
import json

import os
from pathlib import Path

In [2]:
def time_converter(o):
    if isinstance(o, datetime.datetime):
        return o.__str__()

rootpath = Path(os.getcwd())
assert os.path.exists(rootpath), "Output path does not exists."


In [3]:

output_path = os.path.join(rootpath, 'output')
if not os.path.exists(output_path):
    os.mkdir(output_path)
qualification_test_path = os.path.join(output_path, 'qualification_tests')
if not os.path.exists(qualification_test_path):
    os.mkdir(qualification_test_path)
if not os.path.exists(output_path):
    os.mkdir(output_path)
hits_path = os.path.join(output_path,'hits')
if not os.path.exists(hits_path):
    os.mkdir(hits_path)
    
layout_path = os.path.join(rootpath,"layouts" )
assert os.path.exists(layout_path), "Layouts path does not exists. Please check the directory exists."


In [4]:
questions = open(os.path.join(layout_path,'questions.xml'), mode='r').read()
answers = open(os.path.join(layout_path,'answer_key.xml'), mode='r').read()

In [5]:
assert questions and answers, "Either Questions or Answers templates do not exist. Please check and try again!"

region_name = 'us-east-1'

endpoint_url = 'https://mturk-requester-sandbox.us-east-1.amazonaws.com'
#prod_url = "https://mturk-requester.us-east-1.amazonaws.com"

client = boto3.client(
    'mturk',
    endpoint_url=endpoint_url,
    region_name=region_name,
)
# This will return $10,000.00 in the MTurk Developer Sandbox
print(client.get_account_balance()['AvailableBalance'])

10000.00


## Create a new Qualification Test

In [6]:


qual_response = client.create_qualification_type(
                        Name='Vehicle Identification Qualification Test example 1',
                        Keywords='qualification, identify vehicles',
                        Description='qualification test for Vehicle identification',
                        QualificationTypeStatus='Active',
                        Test=questions,
                        AnswerKey=answers,
                        TestDurationInSeconds=600,
                        RetryDelayInSeconds=86400,)
print(qual_response['QualificationType']['QualificationTypeId'])
qual_id = qual_response['QualificationType']['QualificationTypeId']
#print(qual_response)

assert qual_response and qual_id, "Qualification creation error!"

30M5Q3A44UT8HJ21UKYQ253WVTZ7KV


In [7]:
with open(os.path.join(qualification_test_path, str(qual_id)+".json"), 'w') as qualification_json_file:
    json.dump(qual_response, qualification_json_file, default = time_converter)

## Update an existing Qualification Test 

In [11]:
qualids = [qual_id]
for qid in qualids:
    response = client.update_qualification_type(
        QualificationTypeId=qid,
        Test=questions,
        AnswerKey=answers,
        TestDurationInSeconds=600,
        RetryDelayInSeconds=86400,
    )
    print(response['QualificationType']['QualificationTypeId'])

30M5Q3A44UT8HJ21UKYQ253WVTZ7KV


In [12]:
for qid in qualids:
    with open(os.path.join(qualification_test_path, str(qual_id)+".json"), 'w') as qualification_json_file:
        json.dump(response, qualification_json_file, default = time_converter)

## List Qualification Types

In [14]:
response = client.list_qualification_types(
    MustBeRequestable=True,
    MustBeOwnedByCaller=True,
)

In [15]:
response['NumResults']

12

## Delete a apecific Qualification Type Id

In [17]:
qualification_types = client.list_qualification_types(MustBeRequestable=True,MustBeOwnedByCaller=True)['QualificationTypes']

In [18]:
for q in qualification_types:
    if q['QualificationTypeId']==qual_id:
        print(q['QualificationTypeId'])
        hits = client.list_hits_for_qualification_type(QualificationTypeId=q['QualificationTypeId'],)
        if len(hits) == 0:
            response = client.delete_qualification_type(
            QualificationTypeId=q['QualificationTypeId'], 
            )
            print(response)
    if q['QualificationTypeId']==qual_id:
        print(q['QualificationTypeId'])
        hits = client.list_hits_for_qualification_type(QualificationTypeId=q['QualificationTypeId'],)
        if len(hits['HITs']) == 0:
            response = client.delete_qualification_type(
            QualificationTypeId=q['QualificationTypeId'], 
            )
            print(response)

30M5Q3A44UT8HJ21UKYQ253WVTZ7KV
30M5Q3A44UT8HJ21UKYQ253WVTZ7KV
{'ResponseMetadata': {'RequestId': 'e5f32d94-da11-4a61-88e8-8263666e76e9', 'HTTPStatusCode': 200, 'HTTPHeaders': {'x-amzn-requestid': 'e5f32d94-da11-4a61-88e8-8263666e76e9', 'content-type': 'application/x-amz-json-1.1', 'content-length': '2', 'date': 'Tue, 18 May 2021 05:29:05 GMT'}, 'RetryAttempts': 0}}
